In [8]:
#CONFIG INICIAL

import requests
import pandas as pd
import os
import time

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"accept": "application/json"}
RAW_DIR = "dados/raw"
os.makedirs(RAW_DIR, exist_ok=True)

def get_json(url, params=None, retries=3):
    for tentativa in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"[ERRO] Status {response.status_code} - {url}")
        except Exception as e:
            print(f"[EXCEÇÃO] {e}")
        return None

In [13]:
# 1. Deputados
def get_deputados(legislatura):
    print(f"[INICIANDO] Coleta de deputados da legislatura {legislatura}...")
    url = f"{BASE_URL}/deputados"
    data = get_json(url, {"itens": 1000, "idLegislatura": legislatura})
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, f"deputados_legislatura_{legislatura}.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} deputados salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum dado retornado.")
        return pd.DataFrame()

In [15]:
df_deputados = get_deputados(57)
df_deputados.head()

[INICIANDO] Coleta de deputados da legislatura 57...
[FINALIZADO] 673 deputados salvos em 'dados/raw/deputados_legislatura_57.csv'


,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,None
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


In [5]:
# 2. Partidos
def get_partidos():
    print("[INICIANDO] Coleta de partidos...")
    url = f"{BASE_URL}/partidos"
    data = get_json(url)
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, "partidos.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} partidos salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum partido encontrado.")
        return pd.DataFrame()

In [6]:
df_partidos = get_partidos()
df_partidos.head()

[INICIANDO] Coleta de partidos...
[FINALIZADO] 15 partidos salvos em 'dados/raw\partidos.csv'


,id,sigla,nome,uri
0,36898,AVANTE,Avante,https://dadosabertos.camara.leg.br/api/v2/part...
1,37905,CIDADANIA,Cidadania,https://dadosabertos.camara.leg.br/api/v2/part...
2,36899,MDB,Movimento Democrático Brasileiro,https://dadosabertos.camara.leg.br/api/v2/part...
3,37901,NOVO,Partido Novo,https://dadosabertos.camara.leg.br/api/v2/part...
4,36779,PCdoB,Partido Comunista do Brasil,https://dadosabertos.camara.leg.br/api/v2/part...


In [11]:
# 3. Despesas por deputado
def get_despesas_deputados(ano):
    print(f"[INICIANDO] Coleta de despesas dos deputados para o ano {ano}...")

    path_deputados = os.path.join(RAW_DIR, "deputados.csv")
    if not os.path.exists(path_deputados):
        print("[ERRO] Arquivo de deputados não encontrado. Rode primeiro a coleta de deputados.")
        return pd.DataFrame()
    
    deputados_df = pd.read_csv(path_deputados)
    despesas_total = []

    for i, row in deputados_df.iterrows():
        dep_id = row['id']
        dep_nome = row['nome']
        pagina = 1

        while True:
            url = f"{BASE_URL}/deputados/{dep_id}/despesas"
            params = {"ano": ano, "pagina": pagina, "itens": 100}
            data = get_json(url, params)

            if not data or "dados" not in data or not data["dados"]:
                break

            for item in data["dados"]:
                despesas_total.append({
                    "id_deputado": dep_id,
                    "nome_deputado": dep_nome,
                    "tipo_despesa": item.get("tipoDespesa"),
                    "fornecedor": item.get("nomeFornecedor"),
                    "valor": item.get("valorDocumento"),
                    "data": item.get("dataDocumento"),
                    "documento": item.get("numDocumento")
                })

            pagina += 1
            time.sleep(0.02)  # Para evitar sobrecarregar a API

        if (i+1) % 50 == 0:
            print(f"[INFO] Deputado {i+1}/{len(deputados_df)} processado.")

    df_despesas = pd.DataFrame(despesas_total)
    path_out = os.path.join(RAW_DIR, f"despesas_{ano}.csv")
    df_despesas.to_csv(path_out, index=False)
    print(f"[FINALIZADO] {len(df_despesas)} despesas salvas em '{path_out}'")

    return df_despesas

In [12]:
df_despesas = get_despesas_deputados(2022)
df_despesas.head()

[INICIANDO] Coleta de despesas dos deputados para o ano 2022...
[INFO] Deputado 50/513 processado.
[INFO] Deputado 100/513 processado.
[INFO] Deputado 150/513 processado.
[INFO] Deputado 200/513 processado.
[INFO] Deputado 250/513 processado.
[INFO] Deputado 300/513 processado.
[INFO] Deputado 350/513 processado.
[INFO] Deputado 400/513 processado.
[INFO] Deputado 450/513 processado.
[INFO] Deputado 500/513 processado.
[FINALIZADO] 32 despesas salvas em 'dados/raw/despesas_2022.csv'


,id_deputado,nome_deputado,tipo_despesa,fornecedor,valor,data,documento
0,136811,Afonso Hamm,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,LIZIANE QUEVEDO,3090.0,2023-02-17T00:00:00,635
1,136811,Afonso Hamm,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,LIZIANE QUEVEDO,5610.0,2023-02-17T00:00:00,636
2,178835,Afonso Motta,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,NOVA IMPRESSAO GRAFICA E PAPELARIA LTDA,2090.0,2023-02-15T00:00:00,1690
3,109429,Benes Leocádio,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,ELDORADO COMUNICACAO E JORNALISMO LTDA,1500.0,2023-02-09T00:00:00,21
4,204507,Carla Zambelli,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,ALINE DA SILVA SANTOS NASCIMENTO 35683981840,2000.0,2023-02-10T00:00:00,24


In [10]:
# 4. Proposições
def get_proposicoes_por_ano(ano):
    print(f"[INICIANDO] Coleta de proposições para o ano {ano}...")
    todas = []
    pagina = 1

    while True:
        params = {"ano": ano, "itens": 100, "pagina": pagina}
        data = get_json(f"{BASE_URL}/proposicoes", params)

        if not data or "dados" not in data or not data["dados"]:
            break

        todas.extend(data["dados"])
        if pagina % 5 == 0:
            print(f"[INFO] Página {pagina} — total até agora: {len(todas)} proposições")
        pagina += 1
        time.sleep(0.02)

    df = pd.DataFrame(todas)
    path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    df.to_csv(path, index=False)
    print(f"[FINALIZADO] {len(df)} proposições salvas em '{path}'")
    return df


In [11]:
df_proposicoes = get_proposicoes_por_ano(2023)
df_proposicoes.head()


[INICIANDO] Coleta de proposições para o ano 2023...
[INFO] Página 5 — total até agora: 500 proposições
[INFO] Página 10 — total até agora: 1000 proposições
[INFO] Página 15 — total até agora: 1500 proposições
[INFO] Página 20 — total até agora: 2000 proposições
[INFO] Página 25 — total até agora: 2500 proposições
[INFO] Página 30 — total até agora: 3000 proposições
[INFO] Página 35 — total até agora: 3500 proposições
[INFO] Página 40 — total até agora: 4000 proposições
[INFO] Página 45 — total até agora: 4500 proposições
[INFO] Página 50 — total até agora: 5000 proposições
[INFO] Página 55 — total até agora: 5500 proposições
[INFO] Página 60 — total até agora: 6000 proposições
[INFO] Página 65 — total até agora: 6500 proposições
[INFO] Página 70 — total até agora: 7000 proposições
[INFO] Página 75 — total até agora: 7500 proposições
[INFO] Página 80 — total até agora: 8000 proposições
[INFO] Página 85 — total até agora: 8500 proposições
[INFO] Página 90 — total até agora: 9000 proposi

,id,uri,siglaTipo,codTipo,numero,ano,ementa
0,618609,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6155,2023,"Institui o dia 25 de julho como o ""Dia Naciona..."
1,1197773,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,4015,2023,"Altera o art. 121 do Decreto-Lei nº 2.848, de ..."
2,1198010,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2234,2023,Altera redação de dispositivos do artigo 4° da...
3,2074843,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6064,2023,Dispõe sobre direito a dano moral e concessão ...
4,2115253,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2721,2023,Dispõe sobre a prestação de serviços postais a...


AQUI NO MEIO SERIA INTERESSANTE ADICIONAR UM TRATAMENTO NO ARQUIVO proposicoes_2023, POIS A CÉLULA ABAIXO UTILIZA ESSE ARQUIVO COMO INPUT

In [ ]:
# 5. Detalhes proposições
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_votacoes_por_proposicoes_parallel(ano):
    print(f"[INICIANDO] Coleta paralela de votações por proposições do ano {ano}...")
    
    path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    if not os.path.exists(path):
        print("[ERRO] Arquivo de proposições não encontrado.")
        return pd.DataFrame()

    proposicoes = pd.read_csv(path)
    todas_votacoes = []

    def fetch_votacoes(prop_id):
        url = f"{BASE_URL}/proposicoes/{prop_id}/votacoes"
        data = get_json(url)
        if data and "dados" in data:
            for voto in data["dados"]:
                voto["id_proposicao"] = prop_id
            return data["dados"]
        return []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_votacoes, prop_id) for prop_id in proposicoes['id']]
        
        for i, future in enumerate(as_completed(futures), start=1):
            try:
                todas_votacoes.extend(future.result())
            except Exception as e:
                print(f"[ERRO] ao processar futura votação: {e}")
            
            if i % 100 == 0:
                print(f"[INFO] Proposição {i}/{len(proposicoes)} processada")

    df = pd.DataFrame(todas_votacoes)
    path_out = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    df.to_csv(path_out, index=False)
    print(f"[FINALIZADO] {len(df)} votações salvas em '{path_out}'")
    return df


In [3]:
df_votacoes = get_votacoes_por_proposicoes_parallel(2023)
df_votacoes.head()

[INICIANDO] Coleta paralela de votações por proposições do ano 2023...
[INFO] Proposição 100/22900 processada
[INFO] Proposição 200/22900 processada
[INFO] Proposição 300/22900 processada
[INFO] Proposição 400/22900 processada
[INFO] Proposição 500/22900 processada
[INFO] Proposição 600/22900 processada
[INFO] Proposição 700/22900 processada
[INFO] Proposição 800/22900 processada
[INFO] Proposição 900/22900 processada
[INFO] Proposição 1000/22900 processada
[INFO] Proposição 1100/22900 processada
[INFO] Proposição 1200/22900 processada
[INFO] Proposição 1300/22900 processada
[INFO] Proposição 1400/22900 processada
[INFO] Proposição 1500/22900 processada
[INFO] Proposição 1600/22900 processada
[INFO] Proposição 1700/22900 processada
[INFO] Proposição 1800/22900 processada
[INFO] Proposição 1900/22900 processada
[INFO] Proposição 2000/22900 processada
[INFO] Proposição 2100/22900 processada
[INFO] Proposição 2200/22900 processada
[INFO] Proposição 2300/22900 processada
[INFO] Proposição 

,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao,id_proposicao
0,2345493-64,https://dadosabertos.camara.leg.br/api/v2/vota...,2023-05-31,2023-06-01T00:26:58,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Aprovada a Redação Final assinada pelo Relator...,1.0,2345493
1,2345493-62,https://dadosabertos.camara.leg.br/api/v2/vota...,2023-05-31,2023-06-01T00:26:24,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,"Aprovadas as Emenda de Redação nºs 2, 3, 4, 5 ...",1.0,2345493
2,2345493-52,https://dadosabertos.camara.leg.br/api/v2/vota...,2023-05-31,2023-06-01T00:15:40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Suprimido o texto.,NaN,2345493
3,2345493-50,https://dadosabertos.camara.leg.br/api/v2/vota...,2023-05-31,2023-06-01T00:14:11,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Rejeitada a Emenda de Comissão nº 14. Sim: 182...,0.0,2345493
4,2345493-47,https://dadosabertos.camara.leg.br/api/v2/vota...,2023-05-31,2023-05-31T23:56:17,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Mantido o texto. Sim: 260; não: 186; abstenção...,NaN,2345493


In [ ]:
# 6. Votos individuais referentes as proposições
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_votos_individuais_proposicoes(ano, max_workers=10):
    print(f"[INICIANDO] Coleta de votos individuais das proposições de {ano}...")

    path_votacoes = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    if not os.path.exists(path_votacoes):
        print(f"[ERRO] Arquivo votacoes_proposicoes_{ano}.csv não encontrado.")
        return pd.DataFrame()

    votacoes_df = pd.read_csv(path_votacoes)

    # Extrai apenas o número da votação da URI (removendo sufixos como -64)
    votacoes_df["votacao_id"] = votacoes_df["uri"].apply(lambda x: x.split("/")[-1])
    votacao_ids = votacoes_df["votacao_id"].unique()

    def fetch_votos(votacao_id):
        url = f"{BASE_URL}/votacoes/{votacao_id}/votos"
        data = get_json(url)
        if data and "dados" in data:
            df = pd.DataFrame(data["dados"])
            df["id_votacao"] = votacao_id
            return df
        return pd.DataFrame()

    votos_total = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_votos, vid) for vid in votacao_ids]
        for i, future in enumerate(as_completed(futures), 1):
            df = future.result()
            if not df.empty:
                votos_total.append(df)
            if i % 100 == 0:
                print(f"[INFO] Votação {i}/{len(votacao_ids)} processada")

    if votos_total:
        final_df = pd.concat(votos_total, ignore_index=True)
        output_path = os.path.join(RAW_DIR, f"votos_individuais_proposicoes_{ano}.csv")
        final_df.to_csv(output_path, index=False)
        print(f"[FINALIZADO] {len(final_df)} votos salvos em '{output_path}'")
        return final_df
    else:
        print("[AVISO] Nenhum voto encontrado.")
        return pd.DataFrame()


In [3]:
df_votos = get_votos_individuais_proposicoes(2023)
df_votos.head()

[INICIANDO] Coleta de votos individuais das proposições de 2023...
[INFO] Votação 100/7809 processada
[INFO] Votação 200/7809 processada
[INFO] Votação 300/7809 processada
[INFO] Votação 400/7809 processada
[INFO] Votação 500/7809 processada
[INFO] Votação 600/7809 processada
[INFO] Votação 700/7809 processada
[INFO] Votação 800/7809 processada
[INFO] Votação 900/7809 processada
[INFO] Votação 1000/7809 processada
[INFO] Votação 1100/7809 processada
[INFO] Votação 1200/7809 processada
[INFO] Votação 1300/7809 processada
[INFO] Votação 1400/7809 processada
[INFO] Votação 1500/7809 processada
[INFO] Votação 1600/7809 processada
[INFO] Votação 1700/7809 processada
[INFO] Votação 1800/7809 processada
[INFO] Votação 1900/7809 processada
[INFO] Votação 2000/7809 processada
[INFO] Votação 2100/7809 processada
[INFO] Votação 2200/7809 processada
[INFO] Votação 2300/7809 processada
[INFO] Votação 2400/7809 processada
[INFO] Votação 2500/7809 processada
[INFO] Votação 2600/7809 processada
[INFO]

,tipoVoto,dataRegistroVoto,deputado_,id_votacao
0,Sim,2023-06-01T00:12:10,"{'id': 220666, 'uri': 'https://dadosabertos.ca...",2345493-50
1,Não,2023-06-01T00:12:09,"{'id': 178896, 'uri': 'https://dadosabertos.ca...",2345493-50
2,Sim,2023-06-01T00:12:05,"{'id': 220689, 'uri': 'https://dadosabertos.ca...",2345493-50
3,Sim,2023-06-01T00:11:55,"{'id': 178954, 'uri': 'https://dadosabertos.ca...",2345493-50
4,Sim,2023-06-01T00:11:54,"{'id': 204563, 'uri': 'https://dadosabertos.ca...",2345493-50


In [4]:
# 7. Orientações partidárias por votação
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_orientacoes(votacao_id):
    url = f"{BASE_URL}/votacoes/{votacao_id}/orientacoes"
    data = get_json(url)
    orientacoes = []

    if data and "dados" in data:
        for item in data["dados"]:
            item["id_votacao"] = votacao_id
            orientacoes.append(item)

    return orientacoes

def get_orientacoes_por_proposicoes(ano, max_workers=10):
    print(f"[INICIANDO] Coleta de orientações por votação de proposições do ano {ano}...")

    path_votacoes = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    if not os.path.exists(path_votacoes):
        print("[ERRO] Arquivo de votações por proposição não encontrado.")
        return pd.DataFrame()

    votacoes_df = pd.read_csv(path_votacoes)
    votacoes_df["votacao_id"] = votacoes_df["uri"].apply(lambda x: x.split("/")[-1])
    votacao_ids = votacoes_df["votacao_id"].unique().tolist()

    todas_orientacoes = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_orientacoes, vid) for vid in votacao_ids]

        for i, future in enumerate(as_completed(futures), 1):
            orientacoes = future.result()
            todas_orientacoes.extend(orientacoes)
            if i % 100 == 0:
                print(f"[INFO] Votação {i}/{len(votacao_ids)} processada")

    df = pd.DataFrame(todas_orientacoes)
    output_path = os.path.join(RAW_DIR, f"orientacoes_partidarias_proposicoes_{ano}.csv")
    df.to_csv(output_path, index=False)
    print(f"[FINALIZADO] {len(df)} orientações salvas em '{output_path}'")
    return df

In [5]:
df_orientacoes = get_orientacoes_por_proposicoes(2023)
df_orientacoes.head()


[INICIANDO] Coleta de orientações por votação de proposições do ano 2023...
[INFO] Votação 100/7809 processada
[INFO] Votação 200/7809 processada
[INFO] Votação 300/7809 processada
[INFO] Votação 400/7809 processada
[INFO] Votação 500/7809 processada
[INFO] Votação 600/7809 processada
[INFO] Votação 700/7809 processada
[INFO] Votação 800/7809 processada
[INFO] Votação 900/7809 processada
[INFO] Votação 1000/7809 processada
[INFO] Votação 1100/7809 processada
[INFO] Votação 1200/7809 processada
[INFO] Votação 1300/7809 processada
[INFO] Votação 1400/7809 processada
[INFO] Votação 1500/7809 processada
[INFO] Votação 1600/7809 processada
[INFO] Votação 1700/7809 processada
[INFO] Votação 1800/7809 processada
[INFO] Votação 1900/7809 processada
[INFO] Votação 2000/7809 processada
[INFO] Votação 2100/7809 processada
[INFO] Votação 2200/7809 processada
[INFO] Votação 2300/7809 processada
[INFO] Votação 2400/7809 processada
[INFO] Votação 2500/7809 processada
[INFO] Votação 2600/7809 processa

,orientacaoVoto,codTipoLideranca,siglaPartidoBloco,codPartidoBloco,uriPartidoBloco,id_votacao
0,Sim,B,Maioria,NaN,None,2345493-35
1,Sim,B,Fdr PSOL-REDE,NaN,None,2345493-35
2,Sim,B,Bl UniPpFdrPsdbCid...,NaN,None,2345493-35
3,Não,P,PL,37906.0,https://dadosabertos.camara.leg.br/api/v2/part...,2345493-35
4,Sim,B,Fdr PT-PCdoB-PV,NaN,None,2345493-35


In [6]:
# 8. Autores das proposições
def fetch_autores(prop_id):
    url = f"{BASE_URL}/proposicoes/{prop_id}/autores"
    data = get_json(url)
    autores = []

    if data and "dados" in data:
        for item in data["dados"]:
            item["id_proposicao"] = prop_id
            autores.append(item)

    return autores

def get_autores_por_proposicoes(ano, max_workers=10):
    print(f"[INICIANDO] Coleta de autores das proposições do ano {ano}...")

    path_props = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    if not os.path.exists(path_props):
        print("[ERRO] Arquivo de proposições não encontrado.")
        return pd.DataFrame()

    proposicoes_df = pd.read_csv(path_props)
    prop_ids = proposicoes_df["id"].unique().tolist()

    todos_autores = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_autores, pid) for pid in prop_ids]

        for i, future in enumerate(as_completed(futures), 1):
            autores = future.result()
            todos_autores.extend(autores)
            if i % 100 == 0:
                print(f"[INFO] Proposição {i}/{len(prop_ids)} processada")

    df = pd.DataFrame(todos_autores)
    output_path = os.path.join(RAW_DIR, f"autores_proposicoes_{ano}.csv")
    df.to_csv(output_path, index=False)
    print(f"[FINALIZADO] {len(df)} autores salvos em '{output_path}'")
    return df


In [7]:
df_autores = get_autores_por_proposicoes(2023)
df_autores.head()

[INICIANDO] Coleta de autores das proposições do ano 2023...
[INFO] Proposição 100/22900 processada
[INFO] Proposição 200/22900 processada
[INFO] Proposição 300/22900 processada
[INFO] Proposição 400/22900 processada
[INFO] Proposição 500/22900 processada
[INFO] Proposição 600/22900 processada
[INFO] Proposição 700/22900 processada
[INFO] Proposição 800/22900 processada
[INFO] Proposição 900/22900 processada
[INFO] Proposição 1000/22900 processada
[INFO] Proposição 1100/22900 processada
[INFO] Proposição 1200/22900 processada
[INFO] Proposição 1300/22900 processada
[INFO] Proposição 1400/22900 processada
[INFO] Proposição 1500/22900 processada
[INFO] Proposição 1600/22900 processada
[INFO] Proposição 1700/22900 processada
[INFO] Proposição 1800/22900 processada
[INFO] Proposição 1900/22900 processada
[INFO] Proposição 2000/22900 processada
[INFO] Proposição 2100/22900 processada
[INFO] Proposição 2200/22900 processada
[INFO] Proposição 2300/22900 processada
[INFO] Proposição 2400/22900

,uri,nome,codTipo,tipo,ordemAssinatura,proponente,id_proposicao
0,https://dadosabertos.camara.leg.br/api/v2/depu...,Giovani Cherini,10000,Deputado(a),1,1,618609
1,https://dadosabertos.camara.leg.br/api/v2/orga...,Poder Executivo,30000,Órgão do Poder Executivo,1,1,2345493
2,https://dadosabertos.camara.leg.br/api/v2/depu...,Mara Gabrilli,10000,Deputado(a),1,1,2074843
3,https://dadosabertos.camara.leg.br/api/v2/depu...,Efraim Filho,10000,Deputado(a),1,1,1198010
4,https://dadosabertos.camara.leg.br/api/v2/orga...,Poder Executivo,30000,Órgão do Poder Executivo,1,1,2345487


In [8]:
# 9. Temas por proposição
def fetch_temas(prop_id):
    url = f"{BASE_URL}/proposicoes/{prop_id}/temas"
    data = get_json(url)
    temas = []

    if data and "dados" in data:
        for item in data["dados"]:
            item["id_proposicao"] = prop_id
            temas.append(item)

    return temas

def get_temas_por_proposicoes(ano, max_workers=10):
    print(f"[INICIANDO] Coleta de temas das proposições do ano {ano}...")

    path_props = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    if not os.path.exists(path_props):
        print("[ERRO] Arquivo de proposições não encontrado.")
        return pd.DataFrame()

    proposicoes_df = pd.read_csv(path_props)
    prop_ids = proposicoes_df["id"].unique().tolist()

    todos_temas = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_temas, pid) for pid in prop_ids]

        for i, future in enumerate(as_completed(futures), 1):
            temas = future.result()
            todos_temas.extend(temas)
            if i % 100 == 0:
                print(f"[INFO] Proposição {i}/{len(prop_ids)} processada")

    df = pd.DataFrame(todos_temas)
    output_path = os.path.join(RAW_DIR, f"temas_proposicoes_{ano}.csv")
    df.to_csv(output_path, index=False)
    print(f"[FINALIZADO] {len(df)} temas salvos em '{output_path}'")
    return df

In [9]:
df_temas = get_temas_por_proposicoes(2023)
df_temas.head()

[INICIANDO] Coleta de temas das proposições do ano 2023...
[INFO] Proposição 100/22900 processada
[INFO] Proposição 200/22900 processada
[INFO] Proposição 300/22900 processada
[INFO] Proposição 400/22900 processada
[INFO] Proposição 500/22900 processada
[INFO] Proposição 600/22900 processada
[INFO] Proposição 700/22900 processada
[INFO] Proposição 800/22900 processada
[INFO] Proposição 900/22900 processada
[INFO] Proposição 1000/22900 processada
[INFO] Proposição 1100/22900 processada
[INFO] Proposição 1200/22900 processada
[INFO] Proposição 1300/22900 processada
[INFO] Proposição 1400/22900 processada
[INFO] Proposição 1500/22900 processada
[INFO] Proposição 1600/22900 processada
[INFO] Proposição 1700/22900 processada
[INFO] Proposição 1800/22900 processada
[INFO] Proposição 1900/22900 processada
[INFO] Proposição 2000/22900 processada
[INFO] Proposição 2100/22900 processada
[INFO] Proposição 2200/22900 processada
[INFO] Proposição 2300/22900 processada
[INFO] Proposição 2400/22900 p

,codTema,tema,relevancia,id_proposicao
0,34,Administração Pública,0,2345485
1,52,Previdência e Assistência Social,0,2074843
2,56,Saúde,0,2074843
3,57,Defesa e Segurança,0,1198010
4,72,Homenagens e Datas Comemorativas,0,618609
